# SELECT within SELECT

In [1]:
import findspark
import pandas as pd
findspark.init()

SVR = '192.168.31.31'
from pyspark.sql import SparkSession

sc = (SparkSession.builder.appName('app04') 
      .master(f'spark://{SVR}:7077') 
      .config('spark.sql.warehouse.dir', f'hdfs://{SVR}:9000/user/hive/warehouse') 
      .config('spark.cores.max', '4') 
      .config('spark.executor.instances', '1') 
      .config('spark.executor.cores', '2') 
      .config('spark.executor.memory', '10g') 
      .enableHiveSupport().getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Bigger than Russia

**List each country name where the population is larger than that of 'Russia'.**

```
world(name, continent, area, population, gdp)
```

In [2]:
world = sc.read.table('sqlzoo.world')

In [3]:
bm = world.filter(world['name']=='Russia').select('population').collect()[0][0]
world.filter(world['population'] > bm).select('name').toPandas()

,name
0,Bangladesh
1,Brazil
2,China
3,India
4,Indonesia
5,Nigeria
6,Pakistan
7,United States


## 2. Richer than UK

**Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.**

> _Per Capita GDP_   
> The per capita GDP is the gdp/population

In [4]:
a = world.withColumn('pcgdp', world['gdp']/world['population'])
bm = a.filter(a['name']=='United Kingdom').select('pcgdp').head()[0]
a.filter((a['pcgdp']>bm) & (a['continent']=='Europe')).select('name').toPandas()

,name
0,Andorra
1,Austria
2,Belgium
3,Denmark
4,Finland
5,Germany
6,Iceland
7,Ireland
8,Liechtenstein
9,Luxembourg


## 3. Neighbours of Argentina and Australia


List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

In [5]:
bm = [x.continent for x in world.filter(world['name'].isin(
    ['Argentina', 'Australia'])).select('continent').collect()]
world.filter(world['continent'].isin(bm)).select('name', 'continent').toPandas()

,name,continent
0,Argentina,South America
1,Australia,Oceania
2,Bolivia,South America
3,Brazil,South America
4,Chile,South America
5,Colombia,South America
6,Ecuador,South America
7,Fiji,Oceania
8,Guyana,South America
9,Kiribati,Oceania


## 4. Between Canada and Poland

Which country has a population that is more than Canada but less than Poland? Show the name and the population.

In [6]:
popl_can = world.filter(world['name']=='Canada').select('population').head()[0]
popl_pol = world.filter(world['name']=='Poland').select('population').head()[0]
(world.filter((world['population'] > popl_can) & (world['population'] < popl_pol))
 .select('name', 'population').toPandas())

,name,population


## 5. Percentages of Germany

Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

**Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.**

The format should be Name, Percentage for example:

name	| percentage
--------|-----------
Albania	| 3%
Andorra	| 0%
Austria	| 11%
...	| ...

> _Decimal places_   
> You can use the function ROUND to remove the decimal places.

> _Percent symbol %_
> You can use the function CONCAT to add the percentage symbol.

[To get a well rounded view of the important features of SQL you should move on to the next tutorial concerning aggregates.](https://sqlzoo.net/wiki/SUM_and_COUNT)

To gain an absurdly detailed view of one insignificant feature of the language, read on.

We can use the word `ALL` to allow >= or > or < or <=to act over a list. For example, you can find the largest country in the world, by population with this query:

```sql
SELECT name
  FROM world
 WHERE population >= ALL(SELECT population
                           FROM world
                          WHERE population>0)
```

You need the condition **population>0** in the sub-query as some countries have **null** for population.

In [7]:
from pyspark.sql.functions import round, format_string
bm = world.filter(world['name']=='Germany').select('population').head()[0]
(world.filter(world['continent']=='Europe')
    .withColumn('pct', format_string('%d%%', round(100*world['population']/bm, 0).cast('int')))
    .select('name', 'pct').toPandas())

,name,pct
0,Albania,3%
1,Andorra,0%
2,Austria,11%
3,Belarus,11%
4,Belgium,14%
5,Bosnia and Herzegovina,4%
6,Bulgaria,8%
7,Croatia,5%
8,Czech Republic,13%
9,Denmark,7%


## 6. Bigger than every country in Europe

Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

We can refer to values in the outer SELECT within the inner SELECT. We can name the tables so that we can tell the difference between the inner and outer versions.

In [8]:
bm = world.filter(world['continent']=='Europe').agg({'gdp': 'max'}).head()[0]
world.filter(world['gdp']>bm).select('name').toPandas()

,name
0,China
1,Japan
2,United States


## 7. Largest in each continent

**Find the largest country (by area) in each continent, show the continent, the name and the area:**

```sql
SELECT continent, name, population FROM world x
  WHERE population >= ALL
    (SELECT population FROM world y
        WHERE y.continent=x.continent
          AND population>0)
```

> __The above example is known as a correlated or synchronized sub-query.__   
> 
> Using correlated subqueries
> A correlated subquery works like a nested loop: the subquery only has access to rows related to a single > record at a time in the outer query. The technique relies on table aliases to identify two different uses of the same table, one in the outer query and the other in the subquery.
> 
> One way to interpret the line in the **WHERE** clause that references the two table is _“… where the correlated values are the same”._
> 
> In the example provided, you would say _“select the country details from world where the population is greater than or equal to the population of all countries where the continent is the same”._

In [9]:
max_area = (world.groupBy('continent')
            .agg({'area': 'max'})
            .withColumnRenamed('max(area)', 'area'))
(world.join(max_area, on=['continent', 'area'], how='inner')
     .select('continent', 'name', 'area')
     .toPandas())

,continent,name,area
0,Africa,Algeria,2381741.0
1,Oceania,Australia,7692024.0
2,South America,Brazil,8515767.0
3,North America,Canada,9984670.0
4,Asia,China,9596961.0
5,Caribbean,Cuba,109884.0
6,Europe,Kazakhstan,2724900.0
7,"Federated States of,Oceania",Micronesia,702.0
8,Eurasia,Russia,17125242.0


## 8. First country of each continent (alphabetically)

**List each continent and the name of the country that comes first alphabetically.**

In [10]:
(world.groupBy('continent')
     .agg({'name': 'min'})
     .withColumnRenamed('min(name)', 'name')
     .toPandas())

,continent,name
0,"Federated States of,Oceania",Micronesia
1,Africa,Algeria
2,Asia,Afghanistan
3,Caribbean,Antigua and Barbuda
4,Eurasia,Armenia
5,Europe,Albania
6,North America,Belize
7,Oceania,Australia
8,South America,Argentina


## 9. Difficult Questions That Utilize Techniques Not Covered In Prior Sections

**Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.**

In [11]:
bm = [x.continent for x in world.filter(world['population']>2.5e7).select('continent').distinct().collect()]
world.filter(~world['continent'].isin(bm)).select('name', 'continent', 'population').toPandas()

,name,continent,population
0,Antigua and Barbuda,Caribbean,96453.0
1,Bahamas,Caribbean,385340.0
2,Barbados,Caribbean,287025.0
3,Cuba,Caribbean,11209628.0
4,Dominica,Caribbean,71808.0
5,Dominican Republic,Caribbean,10358320.0
6,Grenada,Caribbean,112003.0
7,Haiti,Caribbean,11577779.0
8,Jamaica,Caribbean,2726667.0
9,Micronesia,"Federated States of,Oceania",101351.0


## 10.

**Some countries have populations more than three times that of any of their neighbours (in the same continent). Give the countries and continents.**

In [12]:
from pyspark.sql.functions import isnull
# how many neighbors are there for each continent
bm = world.groupBy('continent').count()
bm = (bm.withColumn('count', bm['count']-1)
         .withColumnRenamed('count', 'nbhrs'))
# compare with tripple population
a = world.join((world
        .withColumn('popl3', 3*world['population'])
        .select('continent', 'name', 'popl3')
        .withColumnRenamed('name', 'name_y')),
    on='continent', how='outer')
b = (a.filter(a['population'] > a['popl3'])
         .select('continent', 'name', 'name_y')
         .groupBy([a.continent, a.name])
         .count()
         .join(bm, on='continent', how='left'))
b.filter(b['count']==b['nbhrs']).select('name', 'continent').toPandas()

,name,continent
0,Brazil,South America
1,Russia,Eurasia


In [13]:
sc.stop()